In [ ]:
#import folder

import pandas as pd
import numpy as np

df = pd.read_csv('../Clean_data/btcfunding_agg.csv')

df["calc_time"] = pd.to_datetime(df["calc_time"], utc=True, errors="coerce")
df["last_funding_rate"] = pd.to_numeric(df["last_funding_rate"], errors="coerce")

df

In [44]:


funding_day = (
    df.set_index("calc_time")
      .resample("1D")
      .agg(
          funding_mean = ("last_funding_rate", "mean"),
          funding_first= ("last_funding_rate", "first"),
          funding_last = ("last_funding_rate", "last"),
          funding_std  = ("last_funding_rate", "std"),   # <- optional
      )
      .reset_index()
      .rename(columns={"calc_time": "date"})
)

z30_funding = funding_day["funding_mean"].rolling(window=30, min_periods=30)
funding_day["z30_funding"] = (funding_day["funding_mean"] - z30_funding.mean()) / z30_funding.std(ddof=0)

z7_funding = funding_day["funding_mean"].rolling(window=7, min_periods=7)
funding_day["z7_funding"] = (funding_day["funding_mean"] - z7_funding.mean()) / z7_funding.std(ddof=0)

funding_day["funding_sign"] = np.sign(funding_day["funding_last"]).astype(int)

funding_day["sign_changed"] = funding_day["funding_sign"].ne(funding_day["funding_sign"].shift(1)).astype(int)
groups = funding_day["sign_changed"].cumsum()

funding_day["funding_streak"] = funding_day.groupby(groups).cumcount() + 1  #how many days in a row has funding stayed the same sign up to today?

funding_day["daily_funding_sum"] = funding_day["funding_mean"] * 3
funding_day["cumfund_7d"] = (
    funding_day["daily_funding_sum"].rolling(window=7, min_periods=7).sum()
)


funding_day["funding_extreme_pos"] = funding_day["z30_funding"] > 2
funding_day["funding_extreme_neg"] = funding_day["z30_funding"] < -2


funding_day.to_csv("../Idea_data/funding_daily.csv")
